In [1]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124/torch_stable.html


Defaulting to user installation because normal site-packages is not writeable

ERROR: Could not find a version that satisfies the requirement torch (from versions: none)
ERROR: No matching distribution found for torch



Looking in indexes: https://download.pytorch.org/whl/cu124/torch_stable.html


In [2]:
!pip install torch===1.4.0 torchvision===0.5.0 -f https://download.pytorch.org/whl/torch_stable.html


Defaulting to user installation because normal site-packages is not writeable
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     -------------------------------------- 796.8/796.8 MB 8.0 MB/s eta 0:00:00
   ---------------------------------------- 1.2/1.2 MB 5.6 MB/s eta 0:00:00
